In [61]:
from os import environ

from sqlalchemy import Connection, create_engine, sql
import pandas as pd
import altair as alt
from pandas import DataFrame
from dotenv import load_dotenv

load_dotenv()

True

In [62]:
def get_database_connection() -> Connection:
    """
    Establishes a database connection to the database specified.
    """

    try:
        engine = create_engine(
            f"mssql+pymssql://{environ['DB_USER']}:{environ['DB_PASSWORD']}@{environ['DB_HOST']}/?charset=utf8")

        return engine.connect()

    except ConnectionError as error:
        print(error)


conn = get_database_connection()

In [63]:
def load_all_plant_data(conn: Connection) -> DataFrame:

    conn.execute(sql.text("USE plants;"))

    query = sql.text(
        """SELECT plant_condition.plant_condition_id,
        plant_condition.at, plant_condition.soil_moisture,
        plant_condition.temperature, plant_condition.last_watered,
        plant.plant_id, plant.plant_name,
        plant.scientific_name, botanist.botanist_id, botanist.first_name,
        botanist.surname, botanist.email, botanist.phone_number,
        origin.origin_id, origin.latitude, origin.longitude, origin.region
        FROM s_epsilon.plant_condition
        JOIN s_epsilon.plant ON s_epsilon.plant.plant_id = s_epsilon.plant_condition.plant_id
        JOIN s_epsilon.botanist ON s_epsilon.botanist.botanist_id = s_epsilon.plant.botanist_id
        JOIN s_epsilon.origin ON s_epsilon.origin.origin_id = s_epsilon.plant.origin_id;""")

    conn.execute(sql.text("COMMIT;"))
    res = conn.execute(query).fetchall()
    df = pd.DataFrame(res)

    return df

df = load_all_plant_data(conn)

df.head(45)

,plant_condition_id,at,soil_moisture,temperature,last_watered,plant_id,plant_name,scientific_name,botanist_id,first_name,surname,email,phone_number,origin_id,latitude,longitude,region
0,1,2023-12-20 15:01:28,96.046952,12.015370,2023-12-20 13:54:32,1,Venus flytrap,N/A,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,2,33.95015,-118.03917,South Whittier
1,2,2023-12-20 15:01:29,97.237929,9.087042,2023-12-20 14:10:54,2,Corpse flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,3,7.65649,4.92235,Efon-Alaaye
2,3,2023-12-20 15:01:30,99.351817,9.964220,2023-12-20 14:50:16,3,Rafflesia arnoldii,N/A,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,1,-19.32556,-41.25528,Resplendor
3,4,2023-12-20 15:01:32,93.730347,11.308067,2023-12-20 13:16:25,4,Black bat flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,4,13.70167,-89.10944,Ilopango
4,5,2023-12-20 15:01:33,99.701431,11.158832,2023-12-20 14:56:18,5,Pitcher plant,Sarracenia catesbaei,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,5,22.88783,84.13864,Jashpurnagar
5,6,2023-12-20 15:01:34,95.218099,10.946320,2023-12-20 13:35:35,6,Wollemi pine,Wollemia nobilis,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,6,43.86682,-79.26630,Markham
6,7,2023-12-20 15:01:36,94.120063,11.487407,2023-12-20 13:23:01,8,Bird of paradise,Heliconia schiedeana 'Fire and Ice',3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,7,5.27247,-3.59625,Bonoua
7,8,2023-12-20 15:01:37,97.164486,10.578360,2023-12-20 14:12:43,9,Cactus,Pereskia grandifolia,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,8,50.98030,11.32903,Weimar
8,9,2023-12-20 15:01:39,93.247665,13.932777,2023-12-20 13:04:57,10,Dragon tree,N/A,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,9,43.50891,16.43915,Split
9,10,2023-12-20 15:01:39,95.266208,9.436539,2023-12-20 13:37:24,11,Asclepias Curassavica,Asclepias curassavica,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,10,20.88953,-156.47432,Kanului


## Average soil moisture

In [64]:
df["soil_moisture"] = df["soil_moisture"].astype(float)

average_soil_moisture = df["soil_moisture"].mean()

average_soil_moisture

93.85866099775286

In [65]:

def get_latest_data(df: DataFrame):
    """
    Returns the latest data of each plant id.
    """
    latest_indices = df.groupby('plant_id')['at'].idxmax()

    latest_conditions = df.loc[latest_indices]

    return latest_conditions



def get_latest_temperature_readings(plants: DataFrame):
    """
    Returns an altair bar chart that shows the latest temperature readings for each plant.
    """
    latest_data = get_latest_data(plants)

    latest_data = latest_data[["plant_name", "temperature"]]
    latest_temp_readings = alt.Chart(latest_data).mark_bar().encode(
        y=alt.Y('plant_name:N'),
        x='temperature:Q',
        tooltip=['plant_name:N', 'temperature:Q'],
        color=alt.Color('plant_name:N', legend=None).scale(scheme='blues')
    ).properties(
        title='Latest Temperature of Plants',
        width=600
    )
    return latest_temp_readings

get_latest_temperature_readings(df)

alt.Chart(...)

In [71]:
unique = df["plant_name"].unique()

unique[0:2]

array(['Venus flytrap', 'Corpse flower'], dtype=object)